<a href="https://colab.research.google.com/github/TnTerry/StockDirection/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 配置环境

In [ ]:
!pip install transformers
!pip install datasets
!pip install tushare

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.8 MB/s eta 0:00:0

# 数据准备




In [ ]:
import pandas as pd
import numpy as np
import torch
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# seq_len = 10
seq_len = 5
return_interval = 1
freq = "D"

stock_news = pd.read_hdf("/content/drive/MyDrive/Thesis/Data/stock_news_df.hdf", key="total_data")

## 数据清洗

In [ ]:
stock_news.dropna(inplace=True) # 去除stock_data为None的新闻数据
stock_news.reset_index(drop=True,inplace=True)
stock_news.shape

(280166, 7)

在从Tushare接口获取数据时，将所有的数据打包存入了字典中

为便于搭建Dataset，将stock_data中的数据展开为dataframe的列

In [ ]:
for var in stock_news['stock_data'][0].keys():
  stock_news[var] = stock_news['stock_data'].apply(lambda x:np.array(x[var]))
stock_news.drop("stock_data",axis=1,inplace=True)
stock_news.head()

,datetime,content,company_name,Date,ts_code,symbol,trade_date,open,high,low,...,vol,amount,turnover_rate,volume_ratio,ma5,ma_v_5,ma10,ma_v_10,ma22,ma_v_22
0,2018-10-25 21:55:02,龙虎榜解读：机构净买这3股 一游资4800万买南京证券,南京证券,20181026,"[601990.SH, 601990.SH, 601990.SH, 601990.SH, 6...",601990,"[20181029, 20181026, 20181025, 20181024, 20181...","[9.23, 9.6, 8.5, 8.44, 8.54, 7.1, 6.71, 6.91, ...","[9.54, 9.85, 9.82, 9.39, 8.54, 7.76, 7.12, 7.0...","[9.09, 9.2, 8.47, 8.31, 8.2, 7.06, 6.65, 6.78,...",...,"[564315.96, 1070854.3, 1278334.02, 1380323.04,...","[524368.637, 1023489.747, 1160560.937, 1215928...","[20.5191, 38.9373, 46.4815, 50.1899, 45.2201, ...","[0.52, 1.14, 1.76, 2.67, 3.82, 1.73, 1.1, 0.72...","[9.16, 8.868, 8.42, 7.816, 7.438, 7.17, 7.028,...","[1107493.856, 1090560.978, 936983.558, 724707....","[8.165, 7.948, 7.755, 7.529, 7.476, 7.46, 7.52...","[716455.638, 683597.694, 605734.706, 513896.01...","[8.36, 8.3591, 8.3595, 8.3177, 8.3127, 8.3423,...","[511153.5573, 507850.5009, 484656.0309, 442063..."
1,2018-10-25 20:54:07,南京证券、西南证券等券商股连日大涨 为各路游资炒作,南京证券,20181026,"[601990.SH, 601990.SH, 601990.SH, 601990.SH, 6...",601990,"[20181029, 20181026, 20181025, 20181024, 20181...","[9.23, 9.6, 8.5, 8.44, 8.54, 7.1, 6.71, 6.91, ...","[9.54, 9.85, 9.82, 9.39, 8.54, 7.76, 7.12, 7.0...","[9.09, 9.2, 8.47, 8.31, 8.2, 7.06, 6.65, 6.78,...",...,"[564315.96, 1070854.3, 1278334.02, 1380323.04,...","[524368.637, 1023489.747, 1160560.937, 1215928...","[20.5191, 38.9373, 46.4815, 50.1899, 45.2201, ...","[0.52, 1.14, 1.76, 2.67, 3.82, 1.73, 1.1, 0.72...","[9.16, 8.868, 8.42, 7.816, 7.438, 7.17, 7.028,...","[1107493.856, 1090560.978, 936983.558, 724707....","[8.165, 7.948, 7.755, 7.529, 7.476, 7.46, 7.52...","[716455.638, 683597.694, 605734.706, 513896.01...","[8.36, 8.3591, 8.3595, 8.3177, 8.3127, 8.3423,...","[511153.5573, 507850.5009, 484656.0309, 442063..."
2,2019-01-06 21:27:25,南京证券：A股将迎来较好战略布局期,南京证券,20190107,"[601990.SH, 601990.SH, 601990.SH, 601990.SH, 6...",601990,"[20190108, 20190107, 20190104, 20190103, 20190...","[9.69, 9.8, 8.75, 8.68, 8.64, 8.54, 9.06, 8.9,...","[10.12, 10.13, 9.79, 9.24, 8.85, 8.93, 9.06, 8...","[9.6, 9.61, 8.69, 8.63, 8.51, 8.54, 8.53, 8.7,...",...,"[518129.47, 785953.42, 844668.42, 362598.82, 1...","[508837.053, 775491.503, 804040.408, 325487.03...","[18.8397, 28.578, 30.713, 13.1845, 6.0563, 8.9...","[1.08, 2.18, 3.72, 1.73, 0.78, 1.21, 0.81, 0.8...","[9.392, 9.17, 8.924, 8.722, 8.74, 8.84, 8.96, ...","[535581.832, 481133.984, 360719.62, 227192.398...","[9.116, 9.065, 9.02, 8.95, 8.974, 9.031, 9.072...","[374689.372, 342583.796, 293181.222, 221866.10...","[9.1964, 9.1714, 9.1595, 9.14, 9.1495, 9.1618,...","[298151.615, 284581.5364, 262360.6909, 242700...."
3,2019-06-13 09:26:37,南京证券今日巨量解禁 开盘大跌9%,南京证券,20190613,"[601990.SH, 601990.SH, 601990.SH, 601990.SH, 6...",601990,"[20190614, 20190613, 20190612, 20190611, 20190...","[9.88, 9.7, 10.65, 10.11, 10.8, 11.13, 11.65, ...","[9.97, 10.35, 10.99, 10.97, 10.8, 11.38, 11.77...","[9.28, 9.7, 10.6, 10.11, 10.01, 10.89, 11.31, ...",...,"[809906.23, 1342488.82, 262197.54, 360633.8, 3...","[777446.259, 1332886.288, 282906.312, 383440.2...","[4.8281, 8.003, 9.5338, 13.113, 10.9825, 8.150...","[1.63, 4.95, 1.01, 1.52, 1.41, 1.08, 0.92, 0.8...","[10.236, 10.55, 10.838, 10.994, 11.162, 11.428...","[615453.564, 498305.638, 271309.002, 258541.59...","[10.832, 11.066, 11.242, 11.316, 11.384, 11.45...","[414616.619, 352820.914, 248816.129, 240754.05...","[11.1032, 11.2009, 11.2732, 11.3282, 11.3509, ...","[286099.6568, 255852.0182, 201100.3073, 203042..."
4,2019-06-17 07:49:58,南京证券上市一年：经纪业务靠天吃饭 投行业务待提高,南京证券,20190617,"[601990.SH, 601990.SH, 601990.SH, 601990.SH, 6...",601990,"[20190618, 20190617, 20190614, 20190613, 20190...","[9.21, 9.38, 9.88, 9.7, 10.65, 10.11, 10.8, 11...","[9.26, 9.43, 9.97, 10.35, 10.99, 10.97, 10.8, ...","[9.02, 9.12, 9.28, 9.7, 10.6, 10.11, 10.01, 10...",...,"[373209.71, 450591.89, 809906.23, 1342488.82, ...","[340557.585, 415434.851, 777446.259, 1332886.2...","[2.2248, 

在dataframe中，所有数据的顺序均为时间逆序。需要重新将其逆置。

同时删除相关无用的数据列，减少内存占用。

In [ ]:
from tqdm.notebook import tqdm

# 删除无关列
stock_news.drop(['ts_code','symbol'],axis=1,inplace=True)

# 设置预测标签
stock_news['pct_chg_label'] = stock_news['pct_chg'].apply(lambda x: np.nan if len(x)==0 else (1 if x[0]>0 else 0))

# 将相关列中的数据调整为时间顺序
inv_col = ['trade_date', 'open', 'high', 'low', 'close', 'pre_close', 'change',
       'pct_chg', 'vol', 'amount', 'turnover_rate', 'volume_ratio', 'ma5',
       'ma_v_5', 'ma10', 'ma_v_10', 'ma22', 'ma_v_22']
for var in tqdm(inv_col,total=len(inv_col)):
  stock_news[var] = stock_news[var].apply(lambda x:x[::-1])

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
stock_news.dropna(inplace=True) # 去除stock_data为None的新闻数据
stock_news.reset_index(drop=True,inplace=True)

将换手率、涨跌幅等数据的量纲换算为标准单位

In [ ]:
stock_news['turnover_rate'] = stock_news['turnover_rate']/100
stock_news['pct_chg'] = stock_news['pct_chg']/100

In [ ]:
spl = int(len(stock_news)*0.8)
print(f"Number of Samples in Train: {spl}") # 训练集样本个数
print(f"Positive Samples in Train: {stock_news['pct_chg_label'][:spl].sum()/spl:.2f}") # 训练集正样本个数
print(f"Number of Samples in Test: {len(stock_news)-spl}") # 测试集样本个数
print(f"Positive Samples in Test: {stock_news['pct_chg_label'][spl:].sum()/(len(stock_news)-spl):.2f}") # 测试集正样本个数

Number of Samples in Train: 218037
Positive Samples in Train: 0.47
Number of Samples in Test: 54510
Positive Samples in Test: 0.49


## 补充生成因子

### Factor1

factor1仿照实习中的因子进行构造：

$$factor1_{i,t} = \frac{close_{i,t}-open_{i,t}}{high_{i,t}-low_{i,t}}$$

In [ ]:
stock_news['f1'] = (stock_news['close']-stock_news['open'])/(stock_news['high']-stock_news['low'])

### Factor2

## 定义Dataset和Dataloader

In [ ]:
import torch
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from multiprocessing import Pool

class MyDataset(Dataset):
  def __init__(self,item,seq_len=seq_len,return_interval=return_interval):
    super(MyDataset,self).__init__()
    self.seq_len = seq_len
    self.return_interval = return_interval

    idx, row = item
    self.row = row
    self.news = row['title']
    # self.factor = row[['f1','turnover_rate']]
    self.factor = np.concatenate([np.nan_to_num(row['f1'],nan=np.nanmean(row['f1'])).reshape(-1,1),
                    np.nan_to_num(row['turnover_rate'],nan=np.nanmean(row['turnover_rate'])).reshape(-1,1)], axis = 1)
    self.label = row['pct_chg_label']

  def __getitem__(self,index):
    return (self.factor,self.news), self.label

  def __len__(self):
    return len(self.row)

with Pool(8) as pool:
  result_datasets = pool.map(MyDataset, stock_news.iterrows())

In [ ]:
train_datasets = ConcatDataset(result_datasets[:int(len(result_datasets)*0.8)])
val_datasets = ConcatDataset(result_datasets[int(len(result_datasets)*0.8):int(len(result_datasets)*0.9)])
test_datasets = ConcatDataset(result_datasets[int(len(result_datasets)*0.9):])

定义Dataloader时需要重新定义collate function，将news经过BERT的tokenizer转化为tensor

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# 自定义处理函数collate_fn，防止批量处理时报错
def col_fn(batch_samples):
    batch_text = []
    batch_label = []
    batch_factor = []
    for X_sample,y_sample in batch_samples:
        batch_text.append(X_sample[1])
        batch_label.append(int(y_sample))
        batch_factor.append(X_sample[0])

    X_news = tokenizer(
        batch_text,
        padding=True, # 默认长度为512，如果超过512进行截断，反之则补至512
        truncation=True,
        return_tensors="pt"
    )

    X_factor = torch.tensor(batch_factor)
    y = torch.tensor(batch_label)
    return (X_news, X_factor), y

batch_size = 32
train_dataloader = DataLoader(train_datasets, batch_size=batch_size, shuffle=True,
                collate_fn=col_fn, drop_last=True)
val_dataloader = DataLoader(val_datasets, batch_size=batch_size, shuffle=True,
                collate_fn=col_fn, drop_last=True)
test_dataloader = DataLoader(test_datasets, batch_size=batch_size, shuffle=True,
                collate_fn=col_fn, drop_last=True)

# 模型搭建

In [ ]:
import torch.nn as nn
from transformers import AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"

class AttentionGRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, seq_len, num_heads,
             output_size, batch_size):
    super(AttentionGRU, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.seq_len = seq_len
    self.batch_size = batch_size
    self.input_size = input_size
    self.output_size = output_size

    self.bert = AutoModel.from_pretrained("bert-base-chinese") # BERT
    self.gru = nn.GRU(input_size, hidden_size, num_layers,
              batch_first = True) # GRU

    self.news_to_h0 = nn.Linear(768,self.hidden_size) # BERT Output to h0
    self.attention = nn.MultiheadAttention(hidden_size, num_heads = num_heads,
                         batch_first = True) # Attention
    self.attention_fc = nn.Linear(seq_len, 1)
    self.output_fc = nn.Linear(hidden_size, output_size) # Output

  def forward(self, news, factors):
    '''
    factors：因子数据，长度为seq_len
    news：经过 tokenizer 处理后的新闻张量
    '''
    bert_outputs = self.bert(**news)
    news = bert_outputs.last_hidden_state[:,0]
    news = self.news_to_h0(news)
    h0 = news.view(-1,self.batch_size,self.hidden_size)
    h0 = torch.cat([h0] * self.num_layers, dim=0)

    x,_ = self.gru(factors,h0) # (batch_size, seq_len, hidden_size)
    x,_ = self.attention(x,x,x) # (batch_size, seq_len, hidden_size)
    x = self.attention_fc(x.permute(0,2,1)) # (batch_size, hidden_size, 1)
    x = self.output_fc(x.squeeze()) # (batch_size, output_size)

    return x

# 模型训练

In [ ]:
from transformers import AdamW, get_scheduler

def train_loop(dataloader, model, optimizer, scheduler, loss_fn):
  # 设置进度条
  # progress_bar = tqdm(range(len(dataloader)))
  # progress_bar.set_description(f'loss:{0:>7f}')
  # finish_batch_num = (epoch-1)*len(dataloader)

  model.train()
  for batch, (X,y) in enumerate(tqdm(dataloader)):
    total_loss = 0
    X_news, X_factor = X
    # X_news, X_factor, y = X_news.to(device), X_factor.to(torch.float32).to(device), y.to(torch.long).to(device)
    X_news, X_factor, y = X_news.to(device), X_factor.to(torch.float32).to(device), y.to(device)
    y_pred = model(X_news, X_factor)
    loss = loss_fn(y_pred.squeeze(0), y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step() # 按照迭代次数降低学习率

    total_loss += loss.item()

  return total_loss


def test_loop(dataloader, model, mode='Test', record=False):
  assert mode in ['Valid', 'Test']
  size = len(dataloader.dataset)
  correct = 0
  model.eval()
  with torch.no_grad():
    count_batch = 0
    false_lst = []
    for idx,(X,y) in enumerate(tqdm(dataloader)):
      X_news, X_factor = X
      X_news, X_factor, y = X_news.to(device), X_factor.to(torch.float32).to(device), y.to(torch.long).to(device)
      y_pred = model(X_news, X_factor)
      correct += (y_pred.squeeze(0).argmax(dim=1) == y).type(torch.float).sum().item()

  correct /= size
  print(f"{mode} Accuracy: {(100*correct):>0.01f}%")
  if record:
    print(false_lst)
  return false_lst

In [ ]:
model = AttentionGRU(input_size=2,hidden_size=16,num_layers=1,num_heads=2,
              seq_len=seq_len,output_size=2,batch_size=batch_size).to(device)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
lr = 1e-4
epoch_num = 3
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=lr)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer = optimizer,
#     num_warmup_steps = 0,
#     num_training_steps = epoch_num*len(train_dataloader)
# )
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optimizer, step_size = 1, gamma = 0.1)

for epoch in range(epoch_num):
  print(f"Epoch {epoch+1} started!")
  total_loss = train_loop(train_dataloader, model, optimizer, lr_scheduler, criterion)
  test_loop(val_dataloader, model,"Valid")
  test_loop(test_dataloader, model,"Test")

Epoch 0 started!


100%|██████████| 135/135 [00:20<00:00,  6.50it/s]


Valid Accuracy: 48.0%


100%|██████████| 135/135 [00:20<00:00,  6.65it/s]


Test Accuracy: 52.9%
Epoch 1 started!


 13%|█▎        | 141/1084 [01:01<06:51,  2.29it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Thesis/model_3label.pt')

# 测试

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Thesis/model_3label.pt'))

In [ ]:
test_loop(test_dataloader, model, mode='Test')